In [1]:
import os.path

import wptools
import pandas as pd
import sys, os

In [2]:
DATA_DIR = '../data/data_processed/' 

In [3]:
old_articles = pd.read_csv(DATA_DIR + 'data_processed.csv', index_col=0)
old_articles.index.names = ['id']

In [7]:
minDate = 1995
articles = old_articles[(old_articles.end_date_clean >= minDate) & (old_articles.categories.str.contains("conflict"))].reset_index().reset_index().rename(columns={'index': 'n_conflict'})

In [8]:
articles

,n_conflict,id,title,categories,article_lenght,views,refs_count,link_count,death,locations,end_date_clean,ongoing
0,0,16760,Kosovo War,military conflict,197476,2368366,274,671,2057,Kosovo Q1231,1999.0,False
1,1,22738,Operation Enduring Freedom,military conflict,52281,1248100,60,228,45000,Afghanistan Q889,2014.0,False
2,2,30770,The Troubles,military conflict,143560,4660829,214,578,1049,Northern Ireland Q26,1998.0,False
3,3,43063,Operation Anaconda,military conflict,53930,428455,34,205,97,Shah- i- Kot Valley Q16205112,2002.0,False
4,4,71717,Second Intifada,military conflict,193420,680777,261,784,2005,Palestinian Authority|Israel,2005.0,False
5,5,140792,First Chechen War,military conflict,63641,985456,66,308,3000,Chechen Republic Q5187|Republic of Ingushe...,1996.0,False
6,6,171392,Second Chechen War,military conflict,134225,1385837,167,634,7321,Chechen Republic Q5187,2009.0,False
7,7,182821,Siege of Sarajevo,military conflict,78135,1060559,85,365,6137,Sarajevo Q11194,1996.0,False
8,8,201936,2003 invasion of Iraq,military conflict,250155,2953828,290,886,551,Iraq Q796,2003.0,False
9,9,202304,Protests against the Iraq War,civil conflict,85504,134088,111,317,0,Global,2002.0,False


#### scegliere quello giusto

In [5]:
articles_to_treat = articles[(articles['n_conflict']<=607)]
#articles_to_treat = articles[(articles['n_conflict']>607) & (articles['n_conflict']<=1214)]
#articles_to_treat = articles[(articles['n_conflict']>1214)]

In [17]:
def get_wiki_military_conflict_deaths(row):
    idPage = row['id']
    page = wptools.page(pageid=idPage)
    
    try:
        page.get_parse()
    except:
        return 0
    
    deaths = None
    
    try: 
        page.get_wikidata()
        deaths = page.data['wikidata']['number of deaths (P1120)']['amount']
        return deaths
    except:
        deaths = None
        
    try:
        deaths = page.data['infobox']['casualties3']
        return deaths
    except:
        deaths = None
        
    
    deaths2 = None
    
    try:
        deaths = page.data['infobox']['casualties1']
    except:
        return None
        
    try:
        deaths2 = page.data['infobox']['casualties2']
        deaths  = deaths + " |caus2| " + deaths2
    except:
        deaths2 = None
        
    if deaths:
        return deaths
    else:
        return row['death']

In [20]:
%%capture
articles_to_treat['death'] = articles_to_treat.apply(get_wiki_military_conflict_deaths,axis=1)

Attenzione a salvare con il nome giusto :)

In [ ]:
articles_to_treat.to_csv(DATA_DIR + 'data_from_wptools_1.csv')

## MERGE

In [ ]:
articles_to_treat = articles_to_treat.set_index('id')

In [ ]:
articles_to_treat = articles_to_treat.drop(['n_conflict'],axis=1)

In [40]:
old_articles.update(articles_to_treat)

In [54]:
articles = articles.reset_index()

In [58]:
articles['id'] = articles.id.astype(int)

In [60]:
articles = articles.set_index('id')

In [67]:
articles = articles.drop(['index'],axis=1)

ok

In [69]:
articles.to_csv(DATA_DIR + 'data_processed_before_death_cleaning.csv')

In [70]:
articles

,title,categories,article_lenght,views,refs_count,link_count,death,locations,end_date_clean,ongoing
id,,,,,,,,,,
16760,Kosovo War,military conflict,197476.0,2368366.0,274.0,671.0,[[File:UCK KLA.png|25px|alt=Logo of the Kosovo...,Kosovo Q1231,1999.0,False
22738,Operation Enduring Freedom,military conflict,52281.0,1248100.0,60.0,228.0,"{{flagicon|Afghanistan}} '''45,000+''' killed<...",Afghanistan Q889,2014.0,False
30770,The Troubles,military conflict,143560.0,4660829.0,214.0,578.0,UDA: 91<br />UVF: 62<br />RHC: 4<br />LVF: 3<b...,Northern Ireland Q26,1998.0,False
43063,Operation Anaconda,military conflict,53930.0,428455.0,34.0,205.0,Unknown number of Afghan fighters killed {{Fla...,Shah- i- Kot Valley Q16205112,2002.0,False
71717,Second Intifada,military conflict,193420.0,680777.0,261.0,784.0,55 foreign citizens total:<br />- 45 foreign c...,Palestinian Authority|Israel,2005.0,False
140792,First Chechen War,military conflict,63641.0,985456.0,66.0,308.0,"30,000–40,000 civilians killed ([[Russian Fede...",Chechen Republic Q5187|Republic of Ingushe...,1996.0,False
171392,Second Chechen War,military conflict,134225.0,1385837.0,167.0,634.0,'''[[Casualties of the Second Chechen War|Civi...,Chechen Republic Q5187,2009.0,False
182821,Siege of Sarajevo,military conflict,78135.0,1060559.0,85.0,365.0,"5,434 civilians killed",Sarajevo Q11194,1996.0,False
201936,2003 invasion of Iraq,military conflict,250155.0,2953828.0,290.0,886.0,Estimated [[Casualties of the Iraq War#Iraqi i...,Iraq Q796,2003.0,False
